In [8]:
import json
import numpy as np
import pandas as pd
import tiktoken

# Path to your file
file_path = "../messages_with_tokens.jsonl"

# Initialize tokenizer for text-embedding-3-small
encoding = tiktoken.encoding_for_model("text-embedding-3-small")

def count_tokens(text: str) -> int:
    if not text:
        return 0
    return len(encoding.encode(text))

# Collect token counts
token_counts = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        try:
            obj = json.loads(line)
        except json.JSONDecodeError:
            continue
        tokens = count_tokens(obj.get("text", ""))
        token_counts.append(tokens)

# Compute statistics
stats = {
    "total_messages": len(token_counts),
    "total_tokens": int(np.sum(token_counts)),
    "min_tokens": int(np.min(token_counts)) if token_counts else 0,
    "max_tokens": int(np.max(token_counts)) if token_counts else 0,
    "mean_tokens": float(np.mean(token_counts)) if token_counts else 0,
    "median_tokens": float(np.median(token_counts)) if token_counts else 0,
    "std_tokens": float(np.std(token_counts)) if token_counts else 0,
    "p25": float(np.percentile(token_counts, 25)) if token_counts else 0,
    "p50": float(np.percentile(token_counts, 50)) if token_counts else 0,
    "p75": float(np.percentile(token_counts, 75)) if token_counts else 0,
    "p90": float(np.percentile(token_counts, 90)) if token_counts else 0,
    "p95": float(np.percentile(token_counts, 95)) if token_counts else 0,
    "p99": float(np.percentile(token_counts, 99)) if token_counts else 0,
}

# Display as a nice dataframe
df_stats = pd.DataFrame([stats])
df_stats


,total_messages,total_tokens,min_tokens,max_tokens,mean_tokens,median_tokens,std_tokens,p25,p50,p75,p90,p95,p99
0,50167,9612852,0,36610,191.617039,7.0,466.536328,0.0,7.0,195.0,720.0,936.0,1523.34


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns  # for nicer style
import plotly.express as px

# Assuming token_counts is already computed (from previous cell)


# --- Plotly version (interactive) ---
fig = px.histogram(
    token_counts, nbins=50,
    title="Distribution of Tokens per Message (Interactive)",
    labels={"value": "Tokens per message", "count": "Number of messages"},
    opacity=0.75
)
fig.update_layout(bargap=0.05)
fig.show()


In [12]:
import plotly.express as px
import plotly.graph_objects as go

MAX_TOKENS = 8192  # embedding model limit

fig = px.histogram(
    token_counts, nbins=50,
    title="Distribution of Tokens per Message (log y-axis, with embedding limit)",
    labels={"value": "Tokens per message", "count": "Number of messages"},
    opacity=0.75
)

# Add vertical line for the token limit
fig.add_vline(
    x=MAX_TOKENS,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Limit = {MAX_TOKENS}",
    annotation_position="top right"
)

# Set y-axis log scale
fig.update_layout(
    yaxis_type="log",
    bargap=0.05
)

fig.show()


In [15]:
import json
import tiktoken
import numpy as np

file_path = "../messages_with_tokens.jsonl"
encoding = tiktoken.encoding_for_model("text-embedding-3-small")

def count_tokens(text: str) -> int:
    if not text:
        return 0
    return len(encoding.encode(text))

# Collect token counts
token_counts = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        try:
            obj = json.loads(line)
        except json.JSONDecodeError:
            continue
        tokens = count_tokens(obj.get("text", ""))
        token_counts.append(tokens)

total_tokens = int(np.sum(token_counts))
num_messages = len(token_counts)

# OpenAI pricing (USD)
price_per_million = 0.02
cost_usd = (total_tokens / 1_000_000) * price_per_million

print(f"Total messages: {num_messages}")
print(f"Total tokens: {total_tokens:,}")
print(f"Estimated embedding cost (text-embedding-3-small): ${cost_usd:.4f} USD")


Total messages: 50167
Total tokens: 9,612,852
Estimated embedding cost (text-embedding-3-small): $0.1923 USD


# Data Formater

In [21]:
# --- Build & REGISTER message_table from your JSONL ---

import json, re
import pandas as pd
from pathlib import Path
import sys

# 👉 Adjust to your repo root so `_data_layer` is importable
sys.path.append(r"c:/Projects/clasificador_mensajes_Copy - Copy")
# sys.path.append(r"c:/Projects/clasificador_mensajes")  # <--

from _data_layer import api  # uses save_artifact/load_artifact under the hood

INPUT = Path(r"C:/Projects/clasificador_mensajes_Copy - Copy/messages_with_tokens.jsonl")  # <-- set your real path
DEFAULT_BATCH = "jsonl_import"

rows = []
with INPUT.open("r", encoding="utf-8") as f:
    for ln in f:
        if not ln.strip():
            continue
        try:
            o = json.loads(ln)
        except json.JSONDecodeError:
            continue

        text = (o.get("text") or "").strip()
        if not text:
            continue

        dataset_id = o.get("dataset_id") or ""
        m = re.search(r"/d{4}-/d{2}-/d{2}", dataset_id)
        batch_id = f"ds_{m.group(0)}" if m else DEFAULT_BATCH

        rows.append({
            "conversation_id": o.get("unit_uid") or "",
            "batch_id":        batch_id,
            "message_id":      o.get("segment_id") or "",
            "sender":          o.get("role") or "",
            "text":            text,
            "timestamp":       o.get("created_at") or "",
        })

df = pd.DataFrame(rows, columns=[
    "conversation_id","batch_id","message_id","sender","text","timestamp"
])

# Basic cleanliness
df["text"] = df["text"].fillna("").astype(str)
df = df.loc[df["text"].str.strip() != ""].reset_index(drop=True)

# --- REGISTER as an artifact so the menu script can load it
batch_ids = sorted(set(df["batch_id"].astype(str).tolist()))
art = api.save_artifact(
    df,
    stage="message_table",                 # 👈 this is what load_artifact("message_table") will look up
    parameters={"source": "jsonl", "n_rows": len(df)},
    backend="parquet",                     # 👈 stored as parquet
    parents=batch_ids                      # optional lineage by batch
)

print("✅ Registered message_table")
print("  id:       ", art.id)
print("  backend:  ", art.backend)
print("  data_ref: ", art.data_ref)

# Quick sanity check (optional): load back the latest "message_table"
df_check, rec = api.load_artifact("message_table")
print("Loaded rows:", len(df_check))


✅ Registered message_table
  id:        artifact_ed267fc3be
  backend:   parquet
  data_ref:  artifacts\parquet\artifact_ed267fc3be.parquet
Loaded rows: 27463
